In [ ]:
# @title PIP
%pip install opencv-python==4.8.0.76
%pip install pytesseract==0.3.9

In [ ]:
from google.colab import drive6
drive.mount('/content/drive')

In [ ]:
# @title TESTES ):D
main()





##Versão nova

In [108]:
import pytesseract
import regex as re
import numpy as np
import cv2
#from google.colab.patches import cv2_imshow
from PIL import ImageFont, ImageDraw, Image
import re
import os
import json

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


def list_files_in_directory(folder_path):
  old_parts = ["primeiroValor","primeiroValor","primeiroValor","primeiroValor"]
  files_array = []
  document_array = []
  for root, dirs, files in os.walk(folder_path):
    for file in files:
        
      file_path = os.path.join(root, file)
      
      parts = file.split('-')
      print(parts)
    
      if parts[0] == old_parts[0] and parts[1] == old_parts[1] and parts[2] == old_parts[2] and parts[3] == old_parts[3]:
        document_array.append(file_path)
      else:
        if old_parts[0] != "primeiroValor":
          files_array.append(document_array)
          document_array = []
        document_array.append(file_path)

      old_parts = parts

  # print(files_array)
  return files_array

def descobre_prestadora(documento):
  if re.findall('^[Cc][Ll][Aa][Rr][Oo]', documento,  re.MULTILINE):
        prestadora = 'Claro'
  elif re.findall('^[Tt][Ii][Mm]', documento,  re.MULTILINE):
        prestadora = 'Tim'
  elif re.findall('^[Vv][Ii][Vv][Oo]', documento, re.MULTILINE):
        prestadora = 'Vivo'
  else:
        prestadora = None

  return prestadora

def extrair_info(documento, prestadora):
  tipo_conta = None
  n_conta = None
  mes_referencia = None
  vencimento = None
  emisao = None
  cpf = None
  periodo = None
  valor_vencimento = None
  valor_vencimento1 = None
  valor_vencimento2 = None
  valor_vencimento3 = None


  
  if prestadora == 'Vivo':
    #PIPELINE VIVO
    tipo_conta = 'Telefonia'

    n_conta = re.findall(r'[Cc][Oo][Nn][Tt][Aa](.*[\r\n].*)(\b\d{9}\b)', documento, re.MULTILINE)

    #TEM VERSÃO COM TEXTO NO DIA DO MES , VERIFICAR ISSO
    #Notas para o paper, esse regex foi complexa de confecionar, a diferença entre o bloco de texto que a geente procurava era "referencia:" ou "referencia "
    mes_referencia = re.findall(r"[Rr][Ee][Ff][Ee][Rr][Êê|Ee][Nn][Cc][Ii][Aa][\:\s|\s](.*?)(0[1-9]|1[0-2])/(19|20)\d{2}", documento, re.MULTILINE)
    #mes_referencia = pre_mes_referencia[1]

    #VENCIMENTO PODE NÃO SER POSSIVEL
    vencimento =  re.findall(r'((.|\n)*?)\b(0[1-9]|1[0-2])\/(19|20[0-9]{2})\b', documento, re.MULTILINE)

    emisao = re.findall(r'[Dd][ÁáAa][Tt][Aa]\s[Dd][Ee] [Ee][Mm][IiÍí][Ss][Ss][ÃãAaÃa][OoOo]\s(0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0,1,2])/(19|20)\d{2}', documento, re.MULTILINE)

    cpf = encontra_cpf_cnpj(documento)

    periodo = re.findall(r'(P|p)(E|e)(R|r)(Í|í)(O|o)(D|d)(O|o)\s\:.*?\n((0[1-9]|[12][0-9]|3[01])\/(0[1-9]|1[0,1,2])\/(19|20)\d{2})', documento, re.MULTILINE)

    valor_vencimento = None


  elif prestadora == 'Claro':

    regex_n_conta = r'[Cc][Oo][Nn][Tt][Aa](.*[\r\n].*)(\b\d{9}\b)'

    # Regex para encontrar a data de vencimento
    regex_data_vencimento = r'\b(\d{2}/\d{2}/\d{4})\b'

    # Regex para encontrar a data de início
    regex_data_inicio = r'\b(\d{2}/\d{2}/\d{4})\b'

    # Regex para encontrar o valor total
    regex_valor_total1 = r'\b(\d+,\d{2})\b'
    regex_valor_total2 = r'TOTAL A PAGAR R\$ (\d{1,3}(?:,\d{3})*(?:\,\d{2})?)'
    regex_valor_total3 = r'Plano Contratado R\$ (\d{1,3}(?:,\d{3})*(?:\,\d{2})?)'

    #PIPELINE CLARO

    tipo_conta = 'Telefonia'



    pre_n_conta = re.findall(regex_n_conta, documento, re.MULTILINE)
    n_conta = pre_n_conta#[0][1]

    pre_mes_referencia = re.findall(regex_data_vencimento, documento, re.MULTILINE)
    mes_referencia = pre_mes_referencia#[0]

    vencimento =  re.findall(r'[Vv][Ee][Nn][Cc][Ii][Mm][Ee][Nn][Tt][Oo]\s(0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0,1,2])/(19|20)\d{2}', documento, re.MULTILINE)

    emisao = re.findall(regex_data_inicio, documento, re.MULTILINE)

    cpf =  re.findall(r'\b\d{3}\.\d{3}\.\d{3}-\d{2}\b', documento, re.MULTILINE)

    periodo = None
    

    # Valor vencimento
    valor_vencimento1 = re.findall(regex_valor_total1, documento, re.MULTILINE)
    valor_vencimento2 = re.findall(regex_valor_total2, documento, re.MULTILINE)
    valor_vencimento3 = re.findall(regex_valor_total3, documento, re.MULTILINE)

    vencimento_numerico1 = [float(elemento.replace(',', '.')) for elemento in valor_vencimento1]
    if vencimento_numerico1:
      valor_vencimento1 = max(vencimento_numerico1)

    vencimento_numerico2 = [float(elemento.replace(',', '.')) for elemento in valor_vencimento2]
    if vencimento_numerico2:
      valor_vencimento2 = max(vencimento_numerico2)

    vencimento_numerico3 = [float(elemento.replace(',', '.')) for elemento in valor_vencimento3]
    if vencimento_numerico3:
      valor_vencimento3 = max(vencimento_numerico3)

    print('teste')
    print(valor_vencimento1)
    print(valor_vencimento2)
    print(valor_vencimento3)
    print('teste')
    if not valor_vencimento2:    
      if not valor_vencimento1:
        valor_vencimento = valor_vencimento3
      else:
        valor_vencimento = valor_vencimento1
    else:
       valor_vencimento = valor_vencimento2 
    

  elif prestadora == 'Tim':
    n_conta = re.findall(r'FATURA: (\d{10})', documento, re.MULTILINE)

    mes_referencia = re.findall(r'(JAN|FEV|MAR|ABR|MAI|JUN|JUL|AGO|SET|OUT|NOV|DEZ)/(19|20)\d{2}', documento, re.MULTILINE)

    vencimento =  re.findall(r'VENCIMENTO\n((0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0-2])/(19|20)\d{2})', documento, re.MULTILINE)

    emisao = re.findall(r'EMISSÃO: ((0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0-2])/(19|20)\d{2})', documento, re.MULTILINE)

    cpf = encontra_cpf_cnpj(documento)

    periodo = re.findall(r'PERÍODO: ((0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0-2])/(19|20)\d{2}) A ((0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0-2])/(19|20)\d{2})', documento, re.MULTILINE)

    #valor_vencimento = None

  nome_conta = re.findall(r'\w+-\d{2}-\d{4}-\w+\.pdf', documento, re.MULTILINE)
  print(str(nome_conta[0]))
  print(prestadora)
  print(tipo_conta)
  print(n_conta)
  print(mes_referencia)
  print(vencimento)
  print(emisao)

  print(periodo)
  print(valor_vencimento)

  print("==============")
  print(cpf)
  # print(valor_vencimento2)
  # print(valor_vencimento3)

  retorno = {
    "prestadora": prestadora,
    "tipo_conta": tipo_conta,
    "n_conta": n_conta,
    "mes_referencia": mes_referencia,
    "vencimento": vencimento,
    "emisao": emisao,
    "cpf": cpf,
    "periodo": periodo,
    "valor_vencimento": valor_vencimento
  }
  # return prestadora, tipo_conta, n_conta, mes_referencia, vencimento, emisao, cpf, periodo, valor_vencimento
  return retorno

def encontra_cpf_cnpj(documento):
  tipo = 'CNPJ'
  dado = re.findall(r"[Cc][Pp][Ff]\/[Cc][Nn][Pp][Jj].*?:.*?(?=\d)(.*?(?=\d)([0-9]{3}\.?[0-9]{3}\.?[0-9]{3}\-?[0-9]{2}))", documento, re.MULTILINE)
  return dado #dado[0][0]

def main(arg):
  fonte = '/content/drive/MyDrive/Colab Notebooks/Fonte/Calibri/Calibri.ttf'


  #paths = ['C:\_CONTAS_COLETADAS\separados\claro-03-2023-johnathan\claro-03-2023-johnathan-page1.png']
  #paths = ['/content/page0.png','/content/page1.png','/content/page2.png']
  #paths = ['/content/drive/MyDrive/_TCC/Contas_coletadas/output-final/vivo-08-2023-cintia-page1.png']
  path = arg
  paths = list_files_in_directory(path)
  documento = ''
  array_documento = []
  for x in paths:
    # documento += str(x)
    # print('------------------')
    # print(x)
    teste = x[0].split('-')
    nome_arquivo = teste[0] + '-' + teste[1] + '-' + teste[2] + '-' + teste[3] + '.pdf'
    # print('=======================')
    # print(nome_arquivo)
    # print('------------------')
    documento = str(nome_arquivo)
    for i, pdfs in enumerate(x):


      img = cv2.imread(x[i])
      convertido = pytesseract.image_to_string(img, lang = 'por', config='--psm 3')
      documento += convertido
      print(documento)
    array_documento.append(documento)
  return array_documento
  #print(array_documento)
  #PEGA PRESTADORA
  
  
  infos = []
  
  for documento in array_documento:
    prestadora = descobre_prestadora(documento)
    info = extrair_info(documento, prestadora)
    infos.append(info)
  print(infos)
  return infos
  # Método de instância para exibir informações da conta


In [ ]:

array_documento = main('C:\_CONTAS_COLETADAS')

# Método de instância para exibir informações da conta

In [ ]:

all_data = []

infos = []

for documento in array_documento:
    prestadora = descobre_prestadora(documento)
    print('+++++++++++++++++++++++++++++++++++++++')
    # print(documento)
    print('+++++++++++++++++++++++++++++++++++++++')
    info = extrair_info(documento, prestadora)
    y = json.dumps(info)
    infos.append(y)
print(infos)

all_data.append(infos)

In [ ]:

import pandas as pd


df = pd.DataFrame(all_data, columns=["prestadora", "tipo_conta", "n_conta", "mes_referencia", "vencimento", "emisao", "cpf", "periodo", "valor_vencimento"])


excel_file = "dados_contas.xlsx"


df.to_excel(excel_file, index=False, engine='openpyxl')

print(f'Dados foram salvos em {excel_file}')